# Missing person and vehicle theft are the most common crimes without arrest resolution in San Francisco

The considered data set consists of crime incidents that have happened during the summer of 2014 in San Francisco. During this analysis, I would like to answer a few questions:
- How many crimes were arrest resolution?
- Which of the most common crimes have poor percentage of positive resolutions (arrests)?
- Is there any relation between day of week and number of crimes?

## Crimes and arrests number per day

On the following chart, the number of crimes and the number of arrests per day was presented.

![Alt text](files/figs/daily_trend.png)
<small><i><a href="SanFranciscoDataAnalysis.ipynb#Crimes-and-arrests-number-per-day">figure source</a></i></small>

As it can be noted, the number of arrests seems to be correlated with the number of crimes. On the other hand there are also days, when the number of crimes increases suddenly, but number of arrests does not change significantly. It may suggests that for some cases, the effectiveness in punishing criminals decreases.

## Crimes number for top 10 the most common categories

In order to find out which of the most common crimes categories have the smallest percentage of arrests, the percentage of arrests for the top 10 most common crimes (with the larger number of crimes) was presented on the following figure.

![Alt text](files/figs/arrests_perc_per_cat.png)
<small><i><a href="SanFranciscoDataAnalysis.ipynb#Crimes-number-for-10-top-common-categories">figure source</a></i></small>

What is very interesting on this figure is that missing person and vehicle theft - crimes which seems to be very 
serious - have very small of percentage of arrests. On the other hand other serious crimes like assault or drug/narcotic are much more often resolve with arrests.

## Number of arrests per week for missing person and vehicle theft

On the last figure, the number of arrests per week was presented for missing person and vehicle theft categories.

![Alt text](files/figs/top_crimes_per_week.png)
<small><i><a href="SanFranciscoDataAnalysis.ipynb#Number-of-arrests-per-week-for-missing-person-and-vehicle-theft">figure source</a></i></small>

As it can be seen, the number of arrests seems to be larger for Sundays in case of vehicle thefts. On the other hand the number of accidents related to missing people increases only during working days. However, it is difficult to discern any patterns of behaviours in this case.